# Projeto 1 - Ciência dos Dados

Nome: Denis Alonso

Nome: Felipe Menke

Nome: Gabriel Correia

___
### Atenção: <br> $\quad$* Serão permitidos grupos de até 3 pessoas, mas com uma rubrica mais exigente. <br> $\quad$* Veja RUBRICA na última página do arquivo PDF que traz o enunciado do Proejto 1.

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\lipem\OneDrive\Documentos\GitHub\PyGAME\Projeto1-Cdados-Insper


___

### Carregando a base de dados com as mensagens dos seus arquivos. <br> Tire o `#` do início da linha de código condizente ao caso escolhido para o projeto 1.

In [4]:
# SE SEU PROJETO USA OS DADOS SOBRE "Airline Passenger Reviews"
train = pd.read_csv('dados_treino_ate_TRIO_FelipeMenke.csv')
test = pd.read_csv('dados_teste_ate_TRIO_FelipeMenke.csv')

In [5]:
# SE SEU PROJETO USA OS DADOS SOBRE "ChatGPT Classification"
#train = pd.read_csv('dados_treino_so_DUPLA_'+nome+'.csv')
#test = pd.read_csv('dados_teste_so_DUPLA_'+nome+'.csv')

In [6]:
train.head()

,Review,Target
0,Singapore to Jakarta. 9 years since I last to...,Passive
1,Norwegian Long Haul. OSL-LGW. Flight DY1310. 1...,Promoter
2,Guangzhou to Paris. I have paid for inflight...,Promoter
3,Vientiane to Melbourne via Bangkok. A very en...,Promoter
4,I travelled Manchester - Sydney return on 777-...,Passive


In [7]:
test.head()

,Review,Target
0,Budapest to Warsaw. The ground crew was very ...,Passive
1,Bergerac to Stansted. Just come off a particul...,Detractor
2,BRU-LIS. Flight left a few minutes late aircra...,Promoter
3,Fleet of A319 and A320 in this route. Fast che...,Passive
4,4 Nov Houston-Doha QR714 and 15 Nov Doha-Houst...,Passive


___
# Classificador Automático (Boot)


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

___
## Montando SEU Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
#Função para limpar o texto


import re 


def cleanup(text):
    
    text = text.replace('\n', ' ').replace('\r', ' ')    
    text = re.sub(r'\s+', ' ', text)
    punctuation = '[´"!-.:?;$'']' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    return text_subbed

**Todas as Avaliações**

In [9]:
#Descobrindo a quantidade total de palavras
dic_total = train.loc[:, "Review"]  
palavras_totais = dic_total.tolist()  
palavras_totais = " ".join(palavras_totais)  
palavras_totais = cleanup(palavras_totais).lower()  
palavras_totais = palavras_totais.split()  
palavras_totais = pd.Series(palavras_totais)  
qtd_palavras_totais = palavras_totais.value_counts()  
qtd_palavras_totais = qtd_palavras_totais.sum() 


**Dados - Avaliações Perfil Passivo**


In [10]:
#PASSIVO

dic_passivos = train.loc[train["Target"]=="Passive","Review"]
#Transformando em Lista
frases_passivos = dic_passivos.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_passivos = " ".join(frases_passivos)
frases_passivos = cleanup(frases_passivos).lower()
frases_passivos = frases_passivos.split()

#Probabilidade de ser Passivo
qtd_palavras_passivos = pd.Series(frases_passivos).value_counts()
qtd_palavras_passivos = qtd_palavras_passivos.sum()

Prob_Passivo = qtd_palavras_passivos/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_passivos = pd.Series(frases_passivos).value_counts(True)
freq_abs_passivos = pd.Series(frases_passivos).value_counts()

**Dados - Avaliações Perfil Detrator**

In [11]:
#DETRACTOR

dic_detrator = train.loc[train["Target"]=="Detractor","Review"]

#Transformando em Lista
frases_detractor = dic_detrator.tolist()
#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_detractor = " ".join(frases_detractor)
frases_detractor = cleanup(frases_detractor).lower()
frases_detractor = frases_detractor.split()

#Probabilidade de ser Detrator
qtd_palavras_detrator = pd.Series(frases_detractor).value_counts()
qtd_palavras_detrator = qtd_palavras_detrator.sum()

Prob_Detrator = qtd_palavras_detrator/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_detrator = pd.Series(frases_detractor).value_counts(True)
freq_abs_detrator = pd.Series(frases_detractor).value_counts()

**Dados - Avaliações Perfil Promoter**

In [12]:
#PROMOTER

dic_promoter = train.loc[train["Target"]=="Promoter","Review"]

#Transformando em Lista
frases_promoter = dic_promoter.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_promoter = " ".join(frases_promoter)
frases_promoter = cleanup(frases_promoter).lower()
frases_promoter = frases_promoter.split()

#Probabilidade de ser Detrator
qtd_palavras_promoter = pd.Series(frases_promoter).value_counts()
qtd_palavras_promoter = qtd_palavras_promoter.sum()

Prob_Promoter = qtd_palavras_promoter/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_promoter = pd.Series(frases_promoter).value_counts(True)
freq_abs_promoter = pd.Series(frases_promoter).value_counts()
print(freq_rel_promoter)

the         0.049208
and         0.037813
to          0.029852
was         0.026083
a           0.021977
              ...   
737800ng    0.000011
sep/13      0.000011
eilna       0.000011
787800      0.000011
dy1310      0.000011
Name: proportion, Length: 6755, dtype: float64


In [30]:
#Criando VOCABULARIO
#Adicionando todas as palavras presentes nas frases uma unica vez:
vocabulario = []

for i in palavras_totais:
    if i not in vocabulario:
        vocabulario.append(i)

print(vocabulario)

qtd_vocabulario = len(vocabulario)
print(qtd_vocabulario)

['singapore', 'to', 'jakarta', '9', 'years', 'since', 'i', 'last', 'took', 'airlines', 'online', 'checkin', 'process', 'easy', 'managed', 'call', 'the', 'hotline', 'change', 'my', 'seat', 'get', 'an', 'exit', 'row', 'with', 'parents', 'which', 'was', 'done', 'easily', 'sq', 'did', 'even', 'bother', 'send', 'us', 'sms', 'warn', 'about', 'french', 'strikes', 'blocking', 'roads', 'and', 'go', 'early', 'airport', 'subsequently', 'very', 'good', 'point', 'find', 'quality', 'of', 'service', 'onboard', 'below', 'that', 'few', 'ago', 'food', 'standard', 'bland', 'staff', 'hardskills', 'though', 'responsive', 'efficient', 'professional', 'but', 'could', 'be', 'better', 'they', 'talk', 'fast', 'robotic', 'gestures', 'more', 'smiling', 'on', 'our', 'inbound', 'leg', 'ran', 'out', 'one', 'option', 'left', 'disappointed', 'seats', 'boeing', '777300er', 'larger', 'comfortable', 'compared', 'other', 'premium', 'for', 'outbound', 'jakartasingapore', 'asked', 'if', 'we', 'wanted', 'take', 'earlier', 'f

___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [31]:
##Codigo da suavização de Laplace

dic_final = {}
Target_test = test.loc[:, "Target"]
Review_test = test.loc[:, "Review"]




for i in range(len(test)):
    tipo = Target_test.iloc[i]  
    frase = cleanup(Review_test.iloc[i]).lower().strip()
    frase = " ".join(frase.split())  


    probDetratordadoFrase = 1
    probPassivodadoFrase = 1
    probPromoterdadoFrase = 1

    for palavra in frase.split():
        palavra = palavra.lower().strip()


        if palavra in frases_promoter:
            probPromoterdadoFrase *= (freq_abs_promoter[palavra]+ 1)/(qtd_palavras_promoter + qtd_vocabulario)  * 1000
        else:
            probPromoterdadoFrase *=(1/(qtd_palavras_promoter + qtd_vocabulario))  * 1000

            
        if palavra in frases_detractor:
            probDetratordadoFrase *= (freq_abs_detrator[palavra]+ 1)/(qtd_palavras_detrator + qtd_vocabulario)   * 1000
        else:
            probDetratordadoFrase *= (1/(qtd_palavras_detrator + qtd_vocabulario)) * 1000

        if palavra in frases_passivos:
            probPassivodadoFrase *= (freq_abs_passivos[palavra]+ 1)/(qtd_palavras_passivos + qtd_vocabulario)   * 1000
        else:
            probPassivodadoFrase *= (1/(qtd_palavras_passivos + qtd_vocabulario))  * 1000

    maior = max(probDetratordadoFrase, probPromoterdadoFrase, probPassivodadoFrase)
    print( probDetratordadoFrase, probPromoterdadoFrase, probPassivodadoFrase)

    if maior == probDetratordadoFrase:
        final = "Detractor"
    elif maior == probPromoterdadoFrase:
        final = "Promoter"
    else:
        final = "Passive"

    dic_final[i] = (tipo, final)


import pandas as pd
df_resultado = pd.DataFrame.from_dict(dic_final, orient="index", columns=["Target", "Predição"])


df_resultado

9666.347548791027 27438954871.118313 768463785.0879236
9.620232611699435e+26 2.8345211907256368e+17 4.071239183355169e+25
37428041.318441644 846222289761031.5 432087194653750.06
13.551515473649344 648180.6029658039 122847470.16127603
1086319.2780372838 72945020530.08113 22290690903462.957
4194203218136.0205 8.141642868171042e+19 1.0738709280431299e+18
66923437.043524444 43107413306.25221 31630579402.854965
1091427126009235.6 6.709293678452009e+17 2.1258697383200417e+25
4.1723560862387976e+16 1705300974904.6577 5.660241364898573e+16
27432675.939307764 81893485.18352792 370314341.70884967
1.5882053613510112e+61 3.5309331780841536e+63 1.5655163033958892e+63
13747539349728.273 8.535014528852476e+17 3.678526029235825e+18
2.3392983963847338e-11 7.500582237164971e-06 1.7149441369782873e-05
1.322754032349451e+18 24494745.48627819 469906.86960277177
1.1507635580772591e+20 1.3412883374328603e+31 1.4862906265854023e+29
417474338.8166057 0.0005391881963386104 970.5404089968766
2100310754.242473 79

,Target,Predição
0,Passive,Promoter
1,Detractor,Detractor
2,Promoter,Promoter
3,Passive,Passive
4,Passive,Passive
...,...,...
1075,Passive,Promoter
1076,Passive,Passive
1077,Detractor,Passive
1078,Detractor,Detractor


In [32]:

Passivo = df_resultado.loc[df_resultado["Target"] == "Passive", "Predição"]
Promoter = df_resultado.loc[df_resultado["Target"] == "Promoter", "Predição"]
Detractor = df_resultado.loc[df_resultado["Target"] == "Detractor", "Predição"]


acerto_passivo = 0
erro_passivo = 0
acerto_promoter = 0
erro_promoter = 0
acerto_detrator = 0
erro_detrator = 0


for idx, predicao in Passivo.items():
    if predicao == "Passive":
        acerto_passivo += 1
    else:
        erro_passivo += 1

for idx, predicao in Promoter.items():
    if predicao == "Promoter":
        acerto_promoter += 1
    else:
        erro_promoter += 1

for idx, predicao in Detractor.items():
    if predicao == "Detractor":
        acerto_detrator += 1
    else:
        erro_detrator += 1


print(f"Passivo: {acerto_passivo} acertos, {erro_passivo} erros")
print(f"Promoter: {acerto_promoter} acertos, {erro_promoter} erros")
print(f"Detractor: {acerto_detrator} acertos, {erro_detrator} erros")


Passivo: 217 acertos, 155 erros
Promoter: 225 acertos, 116 erros
Detractor: 269 acertos, 98 erros


___
## Análise Qualitativa da Performance do Classificador

**Criando a Acurácia dos Preditores**

In [33]:
acur_passivo = (acerto_passivo/(acerto_passivo+erro_passivo))*100
acur_detractor = (acerto_detrator/(acerto_detrator+erro_detrator))*100
acur_promoter = (acerto_promoter/(acerto_promoter+erro_promoter))*100
acur_total = ((acur_promoter+acur_detractor+acur_passivo)/3)
erros_totais = (erro_passivo+erro_detrator+erro_promoter)
acertos_totais = (acerto_passivo+acerto_detrator+acerto_promoter)
erros_totais = (erro_passivo+erro_detrator+erro_promoter)
acertos_totais = (acerto_passivo+acerto_detrator+acerto_promoter)


print(f"O total de acertos do classicador das Targets foi de {acertos_totais}")
print()
print(f"O total de erros do classificador de erros das Targets foi de {erros_totais}")
print()
print(f"A acurácia do classificador da Target Passive é de {acur_passivo:.2f}%")
print()
print(f"A acurácia do classificador da Target Detractor é de {acur_detractor:.2f}%")
print()
print(f"A acurácia do classificador da Target Promoter é de {acur_promoter:.2f}%")
print()



O total de acertos do classicador das Targets foi de 711

O total de erros do classificador de erros das Targets foi de 369

A acurácia do classificador da Target Passive é de 58.33%

A acurácia do classificador da Target Detractor é de 73.30%

A acurácia do classificador da Target Promoter é de 65.98%



In [34]:
acur_total = ((acur_promoter+acur_detractor+acur_passivo)/3)


print(f"A acurácia do classificador de Target foi de {acur_total:.2f}%")

A acurácia do classificador de Target foi de 65.87%


**Análise da quantidade de palavras nas frases de cada Target**

In [35]:
#PASSIVO

dic_passivos_test = test.loc[test["Target"]=="Passive","Review"]
#Transformando em Lista
frases_passivos_test = dic_passivos_test.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_passivos_test = " ".join(frases_passivos_test)
frases_passivos_test = cleanup(frases_passivos_test).lower()
frases_passivos_test = frases_passivos_test.split()

#Probabilidade de ser Passivo
qtd_palavras_passivos_test = pd.Series(frases_passivos_test).value_counts()
qtd_palavras_passivos_test = qtd_palavras_passivos_test.sum()


In [36]:
#DETRACTOR


dic_detrator_test = test.loc[test["Target"]=="Detractor","Review"]

#Transformando em Lista
frases_detractor_test = dic_detrator_test.tolist()
#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_detractor_test = " ".join(frases_detractor_test)
frases_detractor_test = cleanup(frases_detractor_test).lower()
frases_detractor_test = frases_detractor_test.split()

#Probabilidade de ser Detrator
qtd_palavras_detrator_test = pd.Series(frases_detractor_test).value_counts()
qtd_palavras_detrator_test = qtd_palavras_detrator_test.sum()



In [37]:
#PROMOTER

dic_promoter_test = test.loc[test["Target"]=="Promoter","Review"]

#Transformando em Lista
frases_promoter_test = dic_promoter_test.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_promoter_test = " ".join(frases_promoter_test)
frases_promoter_test = cleanup(frases_promoter_test).lower()
frases_promoter_test = frases_promoter_test.split()

#Probabilidade de ser Detrator
qtd_palavras_promoter_test = pd.Series(frases_promoter_test).value_counts()
qtd_palavras_promoter_test = qtd_palavras_promoter_test.sum()

In [38]:
print(f"A quantidade de palavras de cada frases de Passivos é {qtd_palavras_passivos_test}")
print()
print(f"A quantidade de frases de Detrector é {qtd_palavras_detrator_test}")
print()
print(f"A quantidade de frases de Promoter é {qtd_palavras_promoter_test}")
print()

#Quantidades parecidas, o que tem melhor acuracia tem mais palavras porem o que tem a segunda maior acuracia tem a menor acuracia

A quantidade de palavras de cada frases de Passivos é 49898

A quantidade de frases de Detrector é 55514

A quantidade de frases de Promoter é 37523



___
## Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

Refazendo a Limpeza dos textos


In [21]:
import re
import unicodedata
import string

def cleanup2(text):

    text = re.sub(r'http[s]?://\S+', "", text)  
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  
    text = re.sub(r'\s+', " ", text).strip()  
    text = text.lower()  

    text = re.sub(r"\b\d+\b", "", text)  

    text = " ".join([word for word in text.split() if len(word) > 2])
    
    return text 


In [22]:
#Descobrindo a quantidade total de palavras
dic_total2 = train.loc[:, "Review"]  
palavras_totais2 = dic_total2.tolist()  
palavras_totais2 = " ".join(palavras_totais2)  
palavras_totais2 = cleanup2(palavras_totais2).lower()  
palavras_totais2 = palavras_totais2.split()  
palavras_totais2 = pd.Series(palavras_totais2)  
qtd_palavras_totais2 = palavras_totais2.value_counts()  
qtd_palavras_totais2 = qtd_palavras_totais2.sum() 


#PASSIVO

dic_passivos2 = train.loc[train["Target"]=="Passive","Review"]
#Transformando em Lista
frases_passivos2 = dic_passivos2.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_passivos2 = " ".join(frases_passivos2)
frases_passivos2 = cleanup(frases_passivos2).lower()
frases_passivos2 = frases_passivos2.split()

#Probabilidade de ser Passivo
qtd_palavras_passivos2 = pd.Series(frases_passivos2).value_counts()
qtd_palavras_passivos2 = qtd_palavras_passivos2.sum()

Prob_Passivo2 = qtd_palavras_passivos2/qtd_palavras_totais2

#Criando Frequencia Relativa das Palavras

freq_rel_passivos2 = pd.Series(frases_passivos2).value_counts(True)
freq_abs_passivos2 = pd.Series(frases_passivos2).value_counts()

#DETRACTOR

dic_detrator2 = train.loc[train["Target"]=="Detractor","Review"]

#Transformando em Lista
frases_detractor2 = dic_detrator2.tolist()
#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_detractor2 = " ".join(frases_detractor2)
frases_detractor2 = cleanup2(frases_detractor2).lower()
frases_detractor2 = frases_detractor2.split()

#Probabilidade de ser Detrator
qtd_palavras_detrator2 = pd.Series(frases_detractor2).value_counts()
qtd_palavras_detrator2 = qtd_palavras_detrator2.sum()

Prob_Detrator2 = qtd_palavras_detrator2/qtd_palavras_totais2

#Criando Frequencia Relativa das Palavras

freq_rel_detrator2 = pd.Series(frases_detractor2).value_counts(True)
freq_abs_detrator2 = pd.Series(frases_detractor2).value_counts()




#PROMOTER

dic_promoter2 = train.loc[train["Target"]=="Promoter","Review"]

#Transformando em Lista
frases_promoter2 = dic_promoter2.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_promoter2 = " ".join(frases_promoter2)
frases_promoter2 = cleanup2(frases_promoter2).lower()
frases_promoter2 = frases_promoter2.split()

#Probabilidade de ser Detrator
qtd_palavras_promoter2 = pd.Series(frases_promoter2).value_counts()
qtd_palavras_promoter2 = qtd_palavras_promoter2.sum()

Prob_Promoter2 = qtd_palavras_promoter2/qtd_palavras_totais2

#Criando Frequencia Relativa das Palavras

freq_rel_promoter2 = pd.Series(frases_promoter2).value_counts(True)
freq_abs_promoter2 = pd.Series(frases_promoter2).value_counts()

print(freq_rel_promoter2)

the          0.061173
and          0.047007
was          0.032424
flight       0.017842
with         0.014933
               ...   
meaty        0.000013
posting      0.000013
screening    0.000013
kuldoh       0.000013
lhrdohkul    0.000013
Name: proportion, Length: 6355, dtype: float64


**Verificando Performance do Classificador**

In [23]:
##Codigo da suavização de Laplace

dic_final2 = {}
Target_test2 = test.loc[:, "Target"]
Review_test2 = test.loc[:, "Review"]



for i in range(len(test)):
    tipo2 = Target_test2.iloc[i]  
    frase2 = cleanup(Review_test2.iloc[i]).lower().strip()
    frase2 = " ".join(frase2.split())  


    probDetratordadoFrase2 = 1
    probPassivodadoFrase2 = 1
    probPromoterdadoFrase2 = 1

    for palavra2 in frase2.split():
        palavra2 = palavra2.lower().strip()


        if palavra2 in frases_promoter2:
            probPromoterdadoFrase2 *= (freq_abs_promoter2[palavra2]+ 1)/(qtd_palavras_promoter2 + qtd_palavras_totais2)  * 1000
        else:
            probPromoterdadoFrase2 *=(1/(qtd_palavras_promoter2 + qtd_palavras_totais2))  * 1000

            
        if palavra2 in frases_detractor2:
            probDetratordadoFrase2 *= (freq_abs_detrator2[palavra2]+ 1)/(qtd_palavras_detrator2 + qtd_palavras_totais2)   * 1000
        else:
            probDetratordadoFrase2 *= (1/(qtd_palavras_detrator2 + qtd_palavras_totais2)) * 1000

        if palavra2 in frases_passivos2:
            probPassivodadoFrase2 *= (freq_abs_passivos2[palavra2]+ 1)/(qtd_palavras_passivos2 + qtd_palavras_totais2)   * 1000
        else:
            probPassivodadoFrase2 *= (1/(qtd_palavras_passivos2 + qtd_palavras_totais2))  * 1000

    maior2 = max(probDetratordadoFrase2, probPromoterdadoFrase2, probPassivodadoFrase2)

    if maior2 == probDetratordadoFrase2:
        final2 = "Detractor"
    elif maior2 == probPromoterdadoFrase2:
        final2 = "Promoter"
    else:
        final2 = "Passive"

    dic_final2[i] = (tipo2, final2)


import pandas as pd
df_resultado2 = pd.DataFrame.from_dict(dic_final2, orient="index", columns=["Target", "Predição"])


df_resultado2

,Target,Predição
0,Passive,Passive
1,Detractor,Passive
2,Promoter,Passive
3,Passive,Passive
4,Passive,Passive
...,...,...
1075,Passive,Passive
1076,Passive,Passive
1077,Detractor,Passive
1078,Detractor,Passive


In [24]:

Passivo2 = df_resultado2.loc[df_resultado2["Target"] == "Passive", "Predição"]
Promoter2 = df_resultado2.loc[df_resultado2["Target"] == "Promoter", "Predição"]
Detractor2 = df_resultado2.loc[df_resultado2["Target"] == "Detractor", "Predição"]

acerto_passivo2 = 0
erro_passivo2 = 0
acerto_promoter2 = 0
erro_promoter2 = 0
acerto_detrator2 = 0
erro_detrator2 = 0

for idx2, predicao2 in Passivo2.items():
    if predicao2 == "Passive":
        acerto_passivo2 += 1
    else:
        erro_passivo2 += 1

for idx2, predicao2 in Promoter2.items():
    if predicao2 == "Promoter":
        acerto_promoter2 += 1
    else:
        erro_promoter2 += 1

for idx2, predicao2 in Detractor2.items():
    if predicao2 == "Detractor":
        acerto_detrator2 += 1
    else:
        erro_detrator2 += 1


print(f"Passivo: {acerto_passivo2} acertos, {erro_passivo2} erros")
print(f"Promoter: {acerto_promoter2} acertos, {erro_promoter2} erros")
print(f"Detractor: {acerto_detrator2} acertos, {erro_detrator2} erros")


Passivo: 372 acertos, 0 erros
Promoter: 0 acertos, 341 erros
Detractor: 0 acertos, 367 erros


In [25]:
acur_passivo2 = (acerto_passivo2/(acerto_passivo2+erro_passivo2))*100
acur_detractor2 = (acerto_detrator2/(acerto_detrator2+erro_detrator2))*100
acur_promoter2 = (acerto_promoter2/(acerto_promoter2+erro_promoter2))*100
acur_total2 = ((acur_promoter2+acur_detractor2+acur_passivo2)/3)
erros_totais2 = (erro_passivo2+erro_detrator2+erro_promoter2)
acertos_totais2 = (acerto_passivo2+acerto_detrator2+acerto_promoter2)
erros_totais2 = (erro_passivo2+erro_detrator2+erro_promoter2)
acertos_totais2 = (acerto_passivo2+acerto_detrator2+acerto_promoter2)


print(f"O total de acertos do classicador das Targets foi de {acertos_totais2}")
print()
print(f"O total de erros do classificador de erros das Targets foi de erros totais {erros_totais2}")
print()
print(f"A acurácia do classificador da Target Passive é de {acur_passivo2:.2f}%")
print()
print(f"A acurácia do classificador da Target Detractor é de {acur_detractor2:.2f}%")
print()
print(f"A acurácia do classificador da Target Promoter é de {acur_promoter2:.2f}%")
print()

acur_total2 = ((acur_promoter2+acur_detractor2+acur_passivo2)/3)


print(f"A acurácia do classificador de Target foi de {acur_total2:.2f}%")

O total de acertos do classicador das Targets foi de 372

O total de erros do classificador de erros das Targets foi de erros totais 708

A acurácia do classificador da Target Passive é de 100.00%

A acurácia do classificador da Target Detractor é de 0.00%

A acurácia do classificador da Target Promoter é de 0.00%

A acurácia do classificador de Target foi de 33.33%


___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**